# JSSP

### Imports

In [595]:
import pandas as pd
import numpy as np
# import cvxpy as cp
import time
import dimod
import matplotlib.pyplot as plt
from matplotlib import rc
import gurobipy as gp
import math

### Annealing simulation functions

In [596]:
def sendToDwave(qubo, shots=100, beta_range=None):
    # Description: functions that solves a particular qubo problem

    # INPUT:
    # qubo: (matrix) representation of the xt*Q*x problem

    # OUTPUT:
    # sampleset: array of tuples of the form (solution, energy) of length "shots" containing posible (but not neccesarily feasible) solutions
    
    tic = time.perf_counter() # for time measuring
    sampleset = dimod.SimulatedAnnealingSampler().sample_qubo(qubo, num_reads=shots, beta_range=beta_range)
    sampleset = sampleset.aggregate() # solo agrega soluciones DIFERENTES.
    sampleset = [(sample, energy) for sample, energy in zip(sampleset.record.sample, sampleset.record.energy)]
    toc = time.perf_counter() # for time measuring
    print(f"Simulating {shots} instances of annealing took: {(toc-tic)}s")
    # print("Sampleset sin filtrar: ", sampleset)
    return sampleset

def from_sample_to_int(sample):
    binary_list = sample[0].tolist()

    # Convert the binary list to a string and join the elements
    binary_string = ''.join(str(x) for x in binary_list)
    # Use the int() function with base 2 to convert the binary string to an integer
    result = int(binary_string, 2)
    return result

def count_elements(sample_list, total_spaces):
    frequency_dict = {}
    energy_dict = {}
    for item in sample_list:
        hashable_item = from_sample_to_int(item)
        if hashable_item in frequency_dict:
            frequency_dict[hashable_item] += 1
        else:
            energy_dict[hashable_item] = int(item[1])
            frequency_dict[hashable_item] = 1
    results_list = [bin(item)[2:] for item in frequency_dict]
    energies = list(energy_dict.values())
    frequencies = list(frequency_dict.values())

    new_results = []
    for result in results_list:
        if len(result) != total_spaces:
            ceros_string = '0' * int(total_spaces - len(result))
            result = ceros_string + result
        new_results.append(result)
    result = {'results': new_results, 'energies': energies, 'frequencies': frequencies}
    return result

def make_histogram(samples_dict):
    # Constructing an histogram to represent the solutions

    df = pd.DataFrame(samples_dict)
    print(df)

    # Sample data
    data = samples_dict

    # Extract the labels and the amounts into separate lists
    labels = data['results']
    amounts = data['frequencies']
    energies = data['energies']

    # Create a bar plot
    plt.bar(labels, amounts)

    for i, count in enumerate(amounts):
        plt.text(i, count, str(count), ha='center', va='bottom')

    # Add labels and title to the plot
    plt.xlabel('Soluciones')
    plt.ylabel('Cantidades')
    plt.title('Soluciones y cantidad de veces que aparecen')

    # Show the plot
    plt.show()


In [597]:
def lowest_energy(sampleset):  # Finds the lowest energy solution
    
    # Description: given a full sampleset (tuples of the form (solution, energy) finds the lowest energy SAMPLE.
    # INPUTS:
    # Sampleset: a sampleset of the form list((solution, energy))

    # OUTPUTS:
    # best: a tuple of the form (solution, energy)

    if len(sampleset):
        energies = np.array([el[1] for el in sampleset])  # energias
        index = np.argmin(energies)  # indice de la de menor energia
        ret = sampleset[index]  # solucion de menor energia
        return ret

    else:
        return None

### Auxiliary functions

In [598]:
def get_index(v_list, value):
    index = 0
    for l in v_list:
        if l == value:
            return index
        index += 1
    return None


In [599]:
def get_subindexes(item):
    index_list = ""
    for sub_item in item:
        if index_list == "":
            index_list += str(sub_item)
        else:
            index_list += ","
            index_list += str(sub_item)
    return index_list

In [600]:
def get_variable_index(x, var_name):
    try:
        arr = x.index(var_name)
        return arr
    except:
        return None

In [601]:
def add_Q_value(x, q_list, i, j, value):
    index_i = get_variable_index(x, i)
    index_j = get_variable_index(x, j)
    
    if index_i is not None and index_j is not None:
        q_list.append([index_i, index_j, value])


In [602]:

# list1 = []
# x = ["v1", "v2"]
# add_Q_value(x, list1, "v2", "v2", -10)


In [603]:
def update_Q(Q, newQ, p = 1):
    for q in newQ:
        if q[0] == q[1]:
            Q[q[0]][q[0]] += p * q[2]
        else:
            Q[q[0]][q[1]] += p * q[2]
            Q[q[1]][q[0]] += p * q[2]



In [604]:

Q = np.array([[0,0],[0,0]])
print(Q)
newQ = [[0, 0, -1], [0, 1, -2]]
update_Q(Q,newQ)
print(Q)

[[0 0]
 [0 0]]
[[-1 -2]
 [-2  0]]


In [605]:
def get_Q_value(Q, x, i, j):
    index_i = get_variable_index(x, i)
    index_j = get_variable_index(x, j)
    if index_i is not None and index_j is not None:
        return Q[index_i][index_j]
    
    

In [606]:
def print_Q_item(Q, x, i, j):
    # i, j numeric indexes
    if i>=0 and i<len(x) and j>=0 and j<len(x):
        var_i = x[i]
        var_j = x[j]
        print("{", var_i, ",", var_j, "} =", Q[i][j])

In [607]:
def print_Q_nonZero(Q, x):
    for i in range(len(Q)):
        for j in range(len(Q[i])):
            if Q[i][j] != 0:
                print_Q_item(Q, x, i, j)


In [608]:
a = np.matrix([1,2,4,8])


a = np.zeros((1,2))
a[0][1] = 3
q = a.transpose()*a

In [609]:
def linear_constraint(x, vars, a, b=0):

    Q_list = []
    if len(vars) != len(a):
        print("Linear constraint variables error!!!!")
        return Q_list

    a_vector = np.array(a)
    matrix = np.outer(a_vector, a_vector) - 2*b*np.diag(a_vector)

    # print(np.outer(a_vector, a_vector))
    # print(- 2*b*np.diag(a_vector))

    # print(matrix)
    
    for i in range(len(matrix)):
        for j in range(i, len(matrix[i])):
            add_Q_value(x, Q_list, vars[i], vars[j], matrix[i][j])
    
    return Q_list


In [610]:
def print_all_variables(bitvector, x, S_q, L_q, E_q, slack, qb_amount):
    i = 0
    print(len(x))
    while i < len(x):
        if isinstance(x[i], S_q):
            bits = [bitvector[i+j] for j in range(qb_amount)]
            var = x[i].s
            i+=qb_amount
        elif isinstance(x[i], L_q):
            bits = [bitvector[i+j] for j in range(qb_amount)]
            var = x[i].l
            i+=qb_amount
        elif isinstance(x[i], E_q):
            bits = [bitvector[i+j] for j in range(qb_amount)]
            var = x[i].e
            i+=qb_amount
        elif isinstance(x[i], slack):
            slacks = [s for s in x if x[i].same(s)]
            bits = [bitvector[x.index(s)] for s in slacks]
            var = x[i].full_slack_string()
            i+=len(slacks)
        else:
            print(f"{x[i]} = {bitvector[i]}")
            i+=1
            continue
        print(f"{var} = {np.inner(bits, 2**np.arange(len(bits)))}")

## (1) Constants and Definitions



### 1.1) Time slots

T_slots = {1,2,3,... 16} (4 bits: 4qb)

In [611]:
qb_amount = 4

### 1.2) Products
P = {B1, B2, LAM1, LAM2}
              = {0,   1,    2,    3}

In [612]:
class Product:
    def __init__(self, name:str) -> None:
        self.name = name
    
    def __str__(self) -> str:
        return self.name
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.name == __value.name

P = [Product("B1"), Product("B2"), Product("LAM1"), Product("LAM2")]

### 1.3) Jobs
J = {JB1, JB2}

In [613]:
class Job:
    def __init__(self, name:str, final_product: Product = None, amount: int=None, deadline: int=None) -> None:
        self.name = name
        self.final_product = final_product
        self.final_operations = []
        self.amount = amount
        self.deadline = deadline

    def __str__(self) -> str:
        return self.name
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.name == __value.name


J = [Job("JB1"), Job("JB2")]



### 1.4) successors:
      LAM1 -> B1 ---- P2 -> P0
      LAM2 -> B1 ----   P3 -> P0
      LAM1 -> B2 ----   P2 -> P1

In [614]:
successors = [[P[2], P[0]], [P[3], P[0]], [P[2], P[1]]]

### 1.5) amounts:
      a_JB1 = 4
      a_JB2 = 2

In [615]:
J[0].amount = 4
J[1].amount = 2

### 1.6) final product of job:
      JB1 : B1
      JB2 : B2

In [616]:
for j in J:
    j.final_product = P[J.index(j)]


### 1.7) deadline:
      D_JB1 = 10
      D_JB2 = 11

In [617]:
J[0].deadline = 10
J[1].deadline = 11

### 1.8) Machines
M = {M1, M2, M3}

In [618]:
class Machine:
    def __init__(self, name:str) -> None:
        self.name = name
    
    def __str__(self) -> str:
        return self.name
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.name == __value.name

M = [Machine("M1"), Machine("M2"), Machine("M3")]

#### 1.8.1) speeds = 1 for each machine

#### 1.8.2) production machines:

        LAM1, LAM2 -> M1
        B1, B2 -> M2
        B1 -> M3

### 1.9) Operations

      {O_jpms : j=job, i=product, m=machine, s=splitting}

      O_(JB1,LAM1,M1,1)     ->   O_(J0,P2,M0,1) -> O[0]
      O_(JB1,LAM1,M1,2)     ->   O_(J0,P2,M0,2) -> O[1]
      O_(JB1,LAM2,M1,1)     ->   O_(J0,P3,M0,1) -> O[2]
      O_(JB1,LAM2,M1,2)     ->   O_(J0,P3,M0,2) -> O[3]
      O_(JB1,B1,M2,1)       ->   O_(J0,P0,M1,1) -> O[4]
      O_(JB1,B1,M3,1)       ->   O_(J0,P0,M2,1) -> O[5]
      O_(JB2,LAM1,M1,1)     ->   O_(J1,P2,M0,1) -> O[6]
      O_(JB2,B2,M2,1)       ->   O_(J1,P1,M1,1) -> O[7]

In [619]:
class Operation:
    def __init__(self, job: Job, product: Product, machine: Machine, splitting: int=None) -> None:
        self.job = job
        self.product = product
        self.machine = machine
        self.splitting = splitting

    def __str__(self) -> str:
        if self.splitting is not None:
            return f"{self.job},{self.product},{self.machine},{self.splitting}"
        else:
            return f"{self.job},{self.product},{self.machine}"
        
    def to_x_index(self) -> str:
        if self.splitting is not None:
            return f"{self.job},{self.product},{self.splitting}"
        else:
            return f"{self.job},{self.product}"
        
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.job == __value.job and self.product == __value.product and \
            self.machine == __value.machine and (self.splitting is None or self.splitting == __value.splitting)

In [620]:
O = [ Operation(J[0], P[2], M[0], 1),
     Operation(J[0], P[2], M[0], 2), 
     Operation(J[0], P[3], M[0], 1),
     Operation(J[0], P[3], M[0], 2),
     Operation(J[0], P[0], M[1]),      #omit splitting when not needed
     Operation(J[0], P[0], M[2]),
     Operation(J[1], P[2], M[0]),
     Operation(J[1], P[1], M[1])
     ]

In [621]:
# final operations by job

for o in O:
    for j in J:
        if o.job == j and o.product == j.final_product:
            j.final_operations.append(o)


### 1.10) Operations by Machine m:

        OM_m' = {O_jpms : m=m'}
        o_m = #(OM_m)

      M1: {
            O_(JB1,LAM1,M1,1);
            O_(JB1,LAM1,M1,2)
            O_(JB1,LAM2,M1,1)
            O_(JB1,LAM2,M1,2)
            O_(JB2,LAM1,M1,1)
      }
      o_M1 = 5

      M2: {
          O_(JB1,B1,M2,1);
          O_(JB2,B2,M2,1)
      }
      o_M2 = 2

      M3: { O_(JB1,B1,M3,1) }
      o_M3 = 1


In [622]:
OM = []
for m in M:
     count = 0
     op_list = []
     for o in O:
          if o.machine == m:
               op_list.append(o)
     OM.append(op_list)



1.11) (jps), [jpms] : lexicographic order

### 1.12) Transition Cost : TC^m_(i,i')

In [623]:
TC1 = [
     [0,       1500,	0,	1500,	0],
     [200,	0,	     200,	0,	     200],
     [0,	     1500,	0,	1500,	0],
     [200,	0,	     200, 0,	     200],
     [0,	     1500,	0,	1500,	0]
]
TC2 = [
     [0, 200],
     [200, 0]
]
TC3 = [[0]]

TC = [TC1, TC2, TC3]

### 1.13) Transition Time : TT^m_(p,p')

In [624]:
TT1 = [
     [0,	1,	0,	1,	0],
     [0,	0,	0,	0,	0],
     [0,	1,	0,	1,	0],
     [0,	0,	0,	0,	0],
     [0,	1,	0,	1,	0]
]
TT2 = [
     [0, 1],
     [2, 0]
]
TT3 = [[0]]

TT = [TT1, TT2, TT3]

### 1.14) Realizations

      A_JB1,1     = {O_(JB1,LAM1,M1,1); O_(JB1,LAM2,M1,1); O_(JB1,B1,M2,1)}
                  = [O[0], O[2], O[4]]
      A_JB1,2 = {O_(JB1,LAM1,M1,2); O_(JB1,LAM2,M1,2); O_(JB1,B1,M3,1)}
                  = [O[1], O[3], O[5]]
      A_JB2,1 = {O_(JB2,LAM1,M1,1); O_(JB2,B2,M2,1)}
                  = [O[6], O[7]]

      O_(JB1,LAM1,M1,1)     ~   O_(J0,P2,M0,1) ~ O[0]
      O_(JB1,LAM1,M1,2)     ~   O_(J0,P2,M0,2) ~ O[1]
      O_(JB1,LAM2,M1,1)     ~   O_(J0,P3,M0,1) ~ O[2]
      O_(JB1,LAM2,M1,2)     ~   O_(J0,P3,M0,2) ~ O[3]
      O_(JB1,B1,M2,1)       ~   O_(J0,P0,M1,1) ~ O[4]
      O_(JB1,B1,M3,1)       ~   O_(J0,P0,M2,1) ~ O[5]
      O_(JB2,LAM1,M1,1)     ~   O_(J1,P2,M0,1) ~ O[6]
      O_(JB2,B2,M2,1)       ~   O_(J1,P1,M1,1) ~ O[7]


In [625]:
A = [
     [O[0], O[2], O[4]],
     [O[1], O[3], O[5]],
     [O[6], O[7]]
]

1.14.1) Precedences (operations)

      A_JB1,1:
          O_(JB1,LAM1,M1,1) -> O_(JB1,B1,M2,1)
          O_(JB1,LAM2,M1,1) -> O_(JB1,B1,M2,1)
      A_JB1,2:
          O_(JB1,LAM1,M1,2) -> O_(JB1,B1,M3,1)
          O_(JB1,LAM2,M1,2) -> O_(JB1,B1,M3,1)
      A_JB2,1:
          O_(JB2,LAM1,M1,1) -> O_(JB2,B2,M2,1)

In [626]:
prec = [
          [O[0], O[4]],
          [O[2], O[4]],
          [O[1], O[5]],
          [O[3], O[5]],
          [O[6], O[7]],
]

### 1.15) Costs K

1.15.1) K1 = 500 (K0) for both jobs

1.15.2) K2 = 100 (K1) for both jobs

1.15.3) K3 = 1200 (K2) for all machines

1.15.4) K4 = 50 (K3) for all machines

In [627]:
K1 = 500
K2 = 100
K3 = 1200
K4 = 50

In [628]:
index_of_JB1 = get_index(J,"JB2") # ejemplo para obtener el indice de una variable
print(index_of_JB1)

None


## (2) Variables

### Variable Classes

In [629]:
#Start Time
class S:
    def __init__(self, operation: Operation)->None:
        self.operation = operation

    def __str__(self) -> str:
        return f"S{{{self.operation}}}"
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.operation == __value.operation
    
#Start Time qbits
class S_q:
    def __init__(self, s:S, qbit: int) -> None:
        self.s = s
        self.qbit = qbit
    
    def __str__(self) -> str:
        return f"{self.s}_{self.qbit}"

    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.s == __value.s and self.qbit == __value.qbit
    
#Time Interval
class L:
    def __init__(self, operation:Operation)->None:
        self.operation = operation

    def __str__(self) -> str:
        return f"L{{{self.operation}}}"
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.operation == __value.operation
    
#Time Interval qbits
class L_q:
    def __init__(self, l:L, qbit: int) -> None:
        self.l = l
        self.qbit = qbit
    
    def __str__(self) -> str:
        return f"{self.l}_{self.qbit}"

    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.l == __value.l and self.qbit == __value.qbit
    
#Job Finish Time
class E:
    def __init__(self, job: Job) -> None:
        self.job = job
    
    def __str__(self) -> str:
        return f"E{{{self.job}}}"

    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.job == __value.job
    
#Job Finish Time qbits
class E_q:
    def __init__(self, e:E, qbit: int) -> None:
        self.e = e
        self.qbit = qbit
    
    def __str__(self) -> str:
        return f"{self.e}_{self.qbit}"

    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.e == __value.e and self.qbit == __value.qbit

#on time
class d:
    def __init__(self, job: Job) -> None:
        self.job = job

    def __str__(self) -> str:
        return f"d{{{self.job}}}"

    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.job == __value.job
    

#Operation Not Done
class bz:
    def __init__(self, l:L)->None:
        self.l = l

    def __str__(self) -> str:
        return f"bz{{{self.l}}}"
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.l == __value.l

#X Matrix Elements
class X:
    def __init__(self, operation:Operation, k: int)->None:
        self.operation = operation
        self.k = k

    def __str__(self) -> str:
        return f"X^{self.operation.machine}({{{self.operation.to_x_index()}}}, {self.k})"
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.operation == __value.operation and self.k == __value.k
    
#Null column indicator on X^m
class u:
    def __init__(self, machine: Machine, k : int) -> None:
        self.machine = machine
        self.k = k
    
    def __str__(self) -> str:
        return f"u_{{{','.join((str(self.machine), str(self.k)))}}}"

    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.machine == __value.machine and self.k == __value.k
    
#Auxiliary variable for quadratic interactions between X and L o S (name: y if S, z if L)
class AuxQuadXLS:
    def __init__(self,operation:Operation, k: int, qbit:int, name:str)->None:
        self.operation = operation
        self.k = k
        self.qbit = qbit
        self.name = name

    def __str__(self) -> str:
        return f"{self.name}^{self.operation.machine}({{{self.operation.to_x_index()}}}, {self.k})_{self.qbit}"
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.operation == __value.operation and self.k == __value.k\
            and self.name ==__value.name and self.qbit == __value.qbit
    
#Auxiliary variable for quadratic interactions between X'S
class AuxQuadXs:
    def __init__(self, operation1:Operation, operation2:Operation, k:int)->None:
        self.operation1 = operation1
        self.operation2 = operation2
        self.k = k

    def __str__(self) -> str:
        return f"w^{self.operation1.machine}({','.join((self.operation1.to_x_index(), self.operation2.to_x_index(), str(self.k)))})"
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.operation1 == __value.operation1 and \
            self.operation2 == __value.operation2
    
class slack:
    def __init__(self, constraint: str, id: any, qbit:int) -> None:
        self.constraint = constraint
        self.id = id
        self.qbit = qbit

    def __str__(self) -> str:
        return f"slack_{self.constraint}_{self.id}_{self.qbit}"
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.constraint == __value.constraint \
            and self.id == __value.id and self.qbit == __value.qbit
    
    def same(self, __value: object) -> bool:
        return type(__value) == type(self) and self.constraint == __value.constraint \
            and self.id == __value.id

    def full_slack_string(self) -> str:
        return f"slack_{self.constraint}_{self.id}"

In [630]:
x = []  # all variables vector

### 2.1) Start Time: S_jpms (4qb)

In [631]:
S_list = []
for o in O:
    S_list.append(S(o))

2.1.1) qubit digits:

    {S_jpms}_q (binary variable) for each q = 0,1,2,3
    S_jpms = sum_q ( 2^q * {S_jpms}_q )

In [632]:
S_q_list = []
for s in S_list:
    for q in range(qb_amount):
        S_q_list.append(S_q(s,q))

x.extend(S_q_list)

### 2.2) Time Interval: L_jpms (4qb)

In [633]:
L_list = []
for o in O:
    L_list.append(L(o))

2.2.1) qubit digits:

    {L_jpms}_q (binary variable) for each q = 0,1,2,3
    L_jpms = sum_q ( 2^q * {L_jpms}_q )

In [634]:
L_q_list = []
for l in L_list:
    for q in range(qb_amount):
        L_q_list.append(L_q(l, q))

x.extend(L_q_list)

### 2.3) Job finish time / Dummy operation start time:

    E_j (4qb)

In [635]:
E_list = []
for j in J:
    E_list.append(E(j))

2.3.1) qubit digits:

    {E_j}_q (binary variable) for each q = 0,1,2,3
      E_JB1 = sum_q ( 2^q * {E_JB1}_q )
      E_JB2 = S_(JB2,B2,M2,1) + L_(JB2,B2,M2,1)   ((((DEFINED BY OTHER VARIABLES))))


In [636]:
E_q_list = []
for e in E_list:
    for q in range(qb_amount):
        E_q_list.append(E_q(e,q))

x.extend(E_q_list)

### 2.4) Job excess time:

      delta_j^max = E_j - D_j    ((((Not needed, replaceable by E_j -  D_j ))))

### 2.5) "On time" binary variable:

      d_j = ( E_j <= D_j )

In [637]:
d_list = []
for j in J:
    d_list.append(d(j))

x.extend(d_list)

### 2.6) Operation not done / discarded / null operation:

      bz_jpms = ("O_jpms discarded") = (L_jpms == 0)

In [638]:
bz_list = []
for l in L_list:
    bz_list.append(bz(l))
    
x.extend(bz_list)

### 2.7) Null operations amount in machine m:

      ntz_m = sum_jps( bz_jpms )      (((always replaceable by the sum)))

### 2.8) The X matrix: X^m, for each machine m

      X^m_(jpms, k) = ("O_jpms is done in k-th place in machine m")
      X^M1, 5x5       
      X^M2, 2x2
      X^M3, 1x1
      (((( X^m_(jpms, k) = 0 if k > o_m - ntz_m ))))

In [639]:
X_list = []

for m in M:
    for k in range(len(OM[M.index(m)])):
        for o in OM[M.index(m)]:
            X_list.append(X(o, k+1))


x.extend(X_list)

### 2.9) Null column indicator in X^m matrix

      u_mk = ( k <= o_m - ntz_m)

In [640]:
u_list = []
m_index = 0
for m in M:
    for k in range(len(OM[m_index])):
        u_list.append(u(m, k+1))
    m_index += 1

x.extend(u_list)

### 2.10) Auxiliary variables for quadratic constraints

(*) i and j represents a set of indexes (j,p,m,s) of an operation

2.10.1)

    y^m_ikq = X^m_(i,k) * {S_i}_q

2.10.2)

    z^m_ikq = X^m_(i,k) * {L_i}_q

In [641]:
y = []
z = []

m_index = 0
for m in M:
    for o in OM[m_index]:
        for k in range(len(OM[m_index])):
            for q in range(qb_amount):
                y.append(AuxQuadXLS(o,k+1,q,"y"))
                z.append(AuxQuadXLS(o,k+1,q,"z"))
    m_index += 1

# x.extend(y)
# x.extend(z)

2.10.3)

    w^m_ijk = X^m_(i,k) * X^m_(j,k+1)

In [642]:
w = []

m_index = 0
for m in M:
    for o1 in OM[m_index]:
        for o2 in OM[m_index]:
            if len(OM[m_index])>1 and o1 != o2:
                for k in range(len(OM[m_index])):
                    w.append(AuxQuadXs(o1, o2, k+1))
    m_index += 1

# x.extend(w)



### 2.11) Slack variables

In [643]:
for l_q in L_q_list:
    x.append(slack("2", l_q.l, l_q.qbit))

for u_mk in u_list:
    for q in range(math.ceil(math.log2(u_mk.k + 1))):
        x.append(slack("6", u_mk, q))



In [644]:


# print(x)
print(len(x))


167


## (3) Cost: Objective Function (OF)

minimize 3.1) + 3.2) + 3.3) = 

        sum_jpms( (1 - bz_jpms)*(K3_m + K4_m*L_jpms) ) +
        sum_mk,jps,j'i's' ( X^m_jps,k-1 * TC^m_i,i' * X^m_j'i's',k ) +
        sum_j( (1 - d_j) * ( K1_j + K2_j*(E_j - D_j) ) )

In [645]:
#initializar matriz Q
Q = np.zeros((len(x), len(x)))


<h3>3.1) production cost</h3>


In [646]:
Q_FO1 = [] # lista de indices y valor a sumar, ejemplo: [[1,2,16], [1,1,31], [4,3,-10]]
# para cada valor nuevo de Q para la dupla de variables i,j se agrega a la lista de Q_FO1 con add_Q_value(x, q_list, i, j, value)

$$\sum_{jpms}( (1 - bz_{jpms})*(K3_m + K4_m*\sum_{q}(2^q*{L_{jpms}}_q)) ) = $$

  sum_jpms {

      K3_m                   //constant, NOT CONSIDERED IN THE OF

    }


In [647]:
C_FO1 = K3*len(O)   # constant of FO1
print(C_FO1)

9600



  \+ sum_jpms {

      - K3_m * bz_jpms

    }
    
      -> Q_{bz_jpms, bz_jpms} = - K3_m         for all operation jpms

In [648]:
for var in bz_list:
    add_Q_value(x, Q_FO1, var, var, -K3)

\+ sum_jpmsq {
  
    K4_m * 2^q * {L_jpms}_q

  }
  
    -> Q_{L_jpms_q, L_jpms_q} = K4_m * 2^q         for all operation jpms, and bit q


In [649]:
# for lq in L_q:
#     add_Q_value(x, Q_FO1, lq, lq, K4 * 2 ** (int(lq[-1])))

for l in L_list:
    for q in range(qb_amount):
        lq = L_q(l, q)
        add_Q_value(x, Q_FO1, lq, lq, K4 * 2**q)

\+ sum_jpmsq {

    - K4_m * bz_jpms * 2^q * {L_jpms}_q

  }
  
    -> Q_{bz_jpms, L_jpms_q} = -1/2 * K4_m * 2^q         for all operation jpms, and bit q


In [650]:

for i in range(len(O)):
    var = bz_list[i]
    l = L_list[i]
    for q in range(qb_amount):
        add_Q_value(x, Q_FO1, var, L_q(l,q) , -(1/2) * K4 * 2**q)

# print(Q_FO1)
update_Q(Q, Q_FO1)

<h3>3.2) transition cost</h3>

$$\sum_{mk,jps,j'i's'} ( X^{m}_{jps,k-1} * TC^{m}_{i,i'} * X^{m}_{j'i's',k} ) = $$

    sum_m {
      sum_k {
        sum_i,i' {
          TC^m_i,i' * sum_(js)(j's') {
            X^m_(jps, k - 1) * X^m_(j'i's', k)   //quadratic (X^2)
          }
        }
      }
    }

    -> Q_{X^m_(jps, k - 1), X^m_(j'i's', k)} = 1/2 * TC^m_i,i'  
    
         for all m=1,2,3; column k; i,i'=1,2,...8; j,j'=1,2; s,s'=1,2
         
      ~ add_Q_value(x, Q_FO2, X^m_(jps, k-1)), X^m_(j'i's', k), 1/2 * TC^m_i,i')

In [651]:

Q_FO2 = []

m_index = 0
for m in M:
    if len(OM[m_index])>1:
        for k in range(len(OM[m_index])):
            for o1 in OM[m_index]:
                for o2 in OM[m_index]:
                    new_value = (1/2) * TC[m_index] [get_index(P,o1.product)] [get_index(P,o2.product)]
                    if o1 != o2 and new_value != 0:
                        add_Q_value(x,Q_FO2,
                                    X(o1,k),
                                    X(o2,k+1),
                                    new_value)
    m_index += 1
            
update_Q(Q, Q_FO2)

<h3>3.3) out of time cost</h3>

In [652]:
Q_FO3 = []

  $$\sum_{j}( (1 - d_j) * ( K1_j + K2_j*(E_j - D_j) ) ) =$$

    sum_j {
      K1^j - K2^j * D_j       //constant, NOT CONSIDERED IN THE OF
    }


In [653]:
C_FO3 = 0    

for j in J:
    C_FO3 += K1 - K2*j.deadline     # constant of FO3

print(C_FO3)

-1100



  \+ sum_j {

    - ( K1^j - K2^j * D_j ) * d_j   //linear (d_j)

  }
  
      -> Q_{d_j} = K2^j * D_j - K1^j   for all j=1,2



In [654]:
for d_j in d_list:
    job = d_j.job
    new_value = K2 * job.deadline - K1
    add_Q_value(x, Q_FO3, d_j, d_j, new_value)

\+ sum_jq {

    2^q * K2^j * {E_j}_q   //linear (digits of E_j)

  }
  
      -> Q_{E_j_q} = 2^q * K2^j   for all j=1,2; q=0,1,2,3

In [655]:
for e in E_list:
    for q in range(qb_amount):
        new_value = (2**q) * K2
        e_q = E_q(e,q)
        add_Q_value(x, Q_FO3, e_q, e_q, new_value)

\+ sum_jq {

    - 2^q * K2^j * d_j * {E_j}_q    //quadratic (d*E)
    
  }

      -> Q_{d_j, E_j_q} = -1/2 * 2^q * K2^j   for all j=1,2; q=0,1,2,3

In [656]:
for e in E_list:
    d_j = d_list[E_list.index(e)]
    for q in range(qb_amount):
        new_value = -(1/2) * (2**q) * K2
        add_Q_value(x, Q_FO3, d_j, E_q(e,q), new_value)
        # print(Q_FO3)

update_Q(Q, Q_FO3)

print(get_Q_value(Q, x, X(Operation("JB1","LAM1","M1",1),1), X(Operation("JB1","LAM1","M1",2),2)))
print(get_Q_value(Q, x, X(Operation("JB1","LAM1","M1",1),1), X(Operation("JB1","LAM1","M1",1),2)))
print(get_Q_value(Q, x, d("JB2"), d("JB2")))
print(get_variable_index(x, bz_list[0]))

# print_Q_nonZero(Q, x)


None
None
None
74


In [657]:
C_FO = C_FO1 + C_FO3 # constant of FO
print(C_FO)

8500


In [658]:
# sampleset = sendToDwave(Q, 10)
# samples_dict = count_elements(sampleset, len(Q))
# make_histogram(samples_dict)



In [659]:
# for i in x:
#     print(f"{i} = {lowest_energy(sampleset)[0][x.index(i)]}")

In [660]:
# vector=[]
# for i in x:
#     vector.append(1 if (isinstance(i, bz) or isinstance(i, d) or isinstance(i, E_q)) else 0)
# print(vector)
# print(np.sum(vector))
# vector = np.array(vector)
# print(vector.T@Q@vector)
# vector = np.array(lowest_energy(sampleset)[0])
# print(vector.T@Q@vector)

## (4) Constraints



### 4.1) Products demands

#### 4.1.1) Precedent operations

$$ L_{jpms} = \sum_{s'} { L_{jp'm's'}} $$
for all precedent operations O_{jp'm'}
    
    (*) speed = 1 always
    (*) alpha = 1 always (precedent product requirements)

    L_(JB1,B1,M2,1) = L_(JB1,LAM1,M1,1)
    L_(JB1,B1,M2,1) = L_(JB1,LAM2,M1,1)

    L_(JB1,B1,M3,1) = L_(JB1,LAM1,M1,2)
    L_(JB1,B1,M3,1) = L_(JB1,LAM2,M1,2)

    L_(JB2,B2,M2,1) = L_(JB2,LAM1,M1,1)


In [661]:
# Q = np.zeros((len(x), len(x)))
Q_C1 = []
for p in prec:
    
    pre = p[0]
    suc = p[1]
    
    vars = []
    a = []
    for q in range(qb_amount):
        vars.append(L_q(L(suc), q))
        a.append(2**q)
    for q in range(qb_amount):
        vars.append(L_q(L(pre), q))
        a.append(-2**q)

    Q_C1.extend(linear_constraint(x, vars, a))

# print(Q_C1)
P_C1 = 2000
update_Q(Q, Q_C1, P_C1)
# print_Q_nonZero(Q, x)


##### 4.1.1.1) Precedent constraints on variables bz (optional)
  
    bz_i' = 1 if bz_i = 1 for all precedent operations i' -> i

  Penalty: bz_i *(1 - bz_i')
  
    bz_(JB1,B1,M2,1) * (1 - bz_(JB1,LAM1,M1,1) )
    bz_(JB1,B1,M2,1) * (1 - bz_(JB1,LAM2,M1,1) )
    bz_(JB1,B1,M3,1) * (1 - bz_(JB1,LAM1,M1,2) )
    bz_(JB1,B1,M3,1) * (1 - bz_(JB1,LAM2,M1,2) )
    bz_(JB2,B2,M2,1) * (1 - bz_(JB2,LAM1,M1,1) )

#### 4.1.2) Total amount of job

  $$ a_j = \sum_{ms} {L_{j(p_j)ms}} $$
    
  (*) p_j = final product of job j

    a_j = sum_ms {L_j(p_j)ms}
    a_JB1 = L_(JB1,B1,M2,1) + L_(JB1,B1,M3,1)
    a_JB2 = L_(jB2,B2,M2,1)

In [662]:
# Q = np.zeros((len(x), len(x)))

Q_C1_2 = []

for j in J:
    b = j.amount
    vars = []
    a = []
    for of in j.final_operations:
        for q in range(qb_amount):
            vars.append(L_q(L(of), q))
            a.append(2**q)
    
    Q_C1_2.extend(linear_constraint(x, vars, a, b))

P_C1_2 = P_C1
# print(Q_C1_2)
update_Q(Q, Q_C1_2, P_C1_2)
# print_Q_nonZero(Q, x)



### 4.2) Definition of bz

$$ bz_{jpms} = 1\times(L_{jpms} = 0) $$

    bz := (L <= 0)
    (L + Mbz - 1 - slack)^2
    M >= {L_max; 1 - L_min}
         -> M = 16
    slack_max = M - 1 = 15 (4qb)

In [663]:
# (L + M*bz - 1 - slack)^2
M0 = 16
Q_C2 = []

b = 1

for bz_var in bz_list:

    vars = []
    a = []
    
    for q in range(qb_amount):
        vars.append(L_q(bz_var.l, q))
        a.append(2**q)
    
    vars.append(bz_var)
    a.append(M0)

    for q in range(qb_amount):
        vars.append(slack("2", bz_var.l, q))
        a.append(-2**q)
    
    Q_C2.extend(linear_constraint(x, vars, a, b))


# print(Q_C2)
# print(len(Q_C2))
# for q in Q_C2:
#     print(x[q[0]], x[q[1]], q[2])
P_C2 = P_C1
update_Q(Q, Q_C2, P_C2)
# print_Q_nonZero(Q,x)



In [664]:
# Q_coef = 1
# sampleset = sendToDwave(Q_coef * Q, 150, beta_range=(0.01, 7))
# samples_dict = count_elements(sampleset, len(Q))
# make_histogram(samples_dict)

# for i in x:
#     print(f"{i} = {lowest_energy(sampleset)[0][x.index(i)]}")



In [665]:
# print(lowest_energy(sampleset)[1])
# print_all_variables(lowest_energy(sampleset)[0], x, S_q, L_q, E_q, slack, qb_amount)

In [666]:
gp_model = gp.Model()
xv = np.array([[gp_model.addVar(vtype = gp.GRB.BINARY, name = f'x{i}') for i in range(len(x))]])
obj = (xv*np.matrix(Q)*np.transpose(xv))[0,0]
gp_model.params.NonConvex = -1
gp_model.setParam("TimeLimit", 120)
gp_model.setObjective(obj, gp.GRB.MINIMIZE)
gp_model.optimize()
result = np.array([var.x for var in gp_model.getVars()])

Set parameter TimeLimit to value 120
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 0 rows, 167 columns and 0 nonzeros
Model fingerprint: 0x036803c7
Model has 524 quadratic objective terms
Variable types: 0 continuous, 167 integer (167 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [1e+02, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 0.0000000
Found heuristic solution: objective -6450.000000
Presolve removed 0 rows and 95 columns
Presolve time: 0.17s
Presolved: 384 rows, 456 columns, 1152 nonzeros
Variable types: 0 continuous, 456 integer (456 binary)
Found heuristic solution: objective -14650.00000

Root relaxation: objective -4.419150e+06, 37 iterations, 

In [667]:
print((result@np.matrix(Q)@np.transpose(result))[0,0])
print_all_variables(result, x, S_q, L_q, E_q, slack, qb_amount)


-58800.0
167
S{JB1,LAM1,M1,1} = 0.0
S{JB1,LAM1,M1,2} = 0.0
S{JB1,LAM2,M1,1} = 0.0
S{JB1,LAM2,M1,2} = 0.0
S{JB1,B1,M2} = 0.0
S{JB1,B1,M3} = 0.0
S{JB2,LAM1,M1} = 0.0
S{JB2,B2,M2} = 0.0
L{JB1,LAM1,M1,1} = 4.0
L{JB1,LAM1,M1,2} = 0.0
L{JB1,LAM2,M1,1} = 4.0
L{JB1,LAM2,M1,2} = 0.0
L{JB1,B1,M2} = 4.0
L{JB1,B1,M3} = 0.0
L{JB2,LAM1,M1} = 2.0
L{JB2,B2,M2} = 2.0
E{JB1} = 0.0
E{JB2} = 0.0
d{JB1} = 0.0
d{JB2} = 0.0
bz{L{JB1,LAM1,M1,1}} = -0.0
bz{L{JB1,LAM1,M1,2}} = 1.0
bz{L{JB1,LAM2,M1,1}} = 0.0
bz{L{JB1,LAM2,M1,2}} = 1.0
bz{L{JB1,B1,M2}} = -0.0
bz{L{JB1,B1,M3}} = 1.0
bz{L{JB2,LAM1,M1}} = -0.0
bz{L{JB2,B2,M2}} = -0.0
X^M1({JB1,LAM1,1}, 1) = 0.0
X^M1({JB1,LAM1,2}, 1) = 0.0
X^M1({JB1,LAM2,1}, 1) = 0.0
X^M1({JB1,LAM2,2}, 1) = 0.0
X^M1({JB2,LAM1}, 1) = 0.0
X^M1({JB1,LAM1,1}, 2) = 0.0
X^M1({JB1,LAM1,2}, 2) = 0.0
X^M1({JB1,LAM2,1}, 2) = 0.0
X^M1({JB1,LAM2,2}, 2) = 0.0
X^M1({JB2,LAM1}, 2) = 0.0
X^M1({JB1,LAM1,1}, 3) = 0.0
X^M1({JB1,LAM1,2}, 3) = 0.0
X^M1({JB1,LAM2,1}, 3) = 0.0
X^M1({JB1,LAM2,2}, 3) = 0.0
X

  
### 4.3) Definition of d_j

d_j = ( E_j <= D_j )

(delta_j^max not considered)

    E_j - D_j >= 1 - M*d_j
    -> (E_j - D_j + M*d_j - 1 - slack)^2
  con slack <= M - 1      \\\\ con M = 16, alcanza un slack de 4 qb (slack maximo = 15)


In [668]:
# (E_j - D_j + M*d_j - 1 - slack)^2

# FALTA CHEQUEAR !!!!!!
M0 = 16
Q_C3 = []

for e in E_list:
    j = e.job

    b = 1 + j.deadline
    vars = []
    a = []
    
    for q in range(qb_amount):
        vars.append(E_q(e, q))
        a.append(2**q)
    
    vars.append(d(j))
    a.append(M0)

    for q in range(qb_amount):
        vars.append(slack("3", e, q))
        a.append(-2**q)
    
    Q_C3.extend(linear_constraint(x, vars, a, b))


P_C3 = 2100
# update_Q(Q, Q_C3, P_C3)



### 4.4) Definition of ntz_m

    ntz_m = sum_jps( bz_jpms )  (((((( not necessary ))))))



<h3>4.5) Definition of X^m matrices.</h3>
Null row for each null operation

    sum_k ( X^m_(jps,k) ) = 1 - bz_jpms


In [669]:
# (sum_k ( X^m_(jps,k) ) + bz_jpms - 1)^2
# Q = np.zeros((len(x), len(x)))

Q_C5 = []

for o in O:

    b = 1
    vars = []
    a = []
    
    for k in range(len(OM[M.index(o.machine)])):
        vars.append(X(o, k+1))
        a.append(1)

    vars.append(bz(L(o)))
    a.append(1)
    
    Q_C5.extend(linear_constraint(x, vars, a, b))


P_C5 = 2000
# print(Q_C5)
# update_Q(Q, Q_C5, P_C5)
# print_Q_nonZero(Q,x)



### 4.6) Definition of u_mk

$$ u_{mk} = 1\times( k <= o_m - \sum_{jps} {bz_{jpms}} ) $$

    for all m and k
    
      k - o_m + sum(bz) >= - M * u_mk + 1
      -> (k - o_m + sum(bz) + M * u_mk - 1 - slack)^2
      with slack_max = M - 1 >= max{k - o_m + sum(bz)} = k - o_m + max{sum(bz)} = k - o_m + o_m = k

      slack_max = M - 1 >= k

      * M1: o_m = 5; k = 1,2,3,4,5;
        k = 1: slack de 1qb, M = 2
        k = 2,3: slack de 2qb, M = 4
        k = 4,5: slack de 3qb, M = 8

        (sum(bz) + 2 * u_(M1,1) - 5 - slack_0)^2
        (sum(bz) + 4 * u_(M1,2) - 4 - slack_0 - 2*slack_1)^2
        (sum(bz) + 4 * u_(M1,3) - 3 - slack_0 - 2*slack_1)^2
        (sum(bz) + 8 * u_(M1,4) - 2 - slack_0 - 2*slack_1 - 4*slack_2)^2
        (sum(bz) + 8 * u_(M1,5) - 1 - slack_0 - 2*slack_1 - 4*slack_2)^2
        
        (*) sum(bz) = bz_(JB1,LAM1,M1,1) + bz_(JB1,LAM1,M1,2) + bz_(JB1,LAM2,M1,1) + bz_(JB1,LAM2,M1,2) + bz_(JB2,LAM1,M1,1)

      * M2: o_m = 2; k = 1,2;
        k = 1: slack de 1qb, M = 2
        k = 2: slack de 2qb, M = 4

        (sum(bz) + 2 * u_mk - 2 - slack_0)^2
        (sum(bz) + 4 * u_mk - 1 - slack_0 - 2*slack_1)^2

      * M3: o_m = 1; k = 1
        u_(M3,1) = 1 - bz_(JB1,B1,M3,1)



In [670]:
Q_C6 = []

# for all m and k    
# (k - o_m + sum(bz) + M * u_mk - 1 - slack)^2
# slack_max = M - 1 >= k
# slack de n qubits:
#   M = 2^n
#   n = techo{log2{k+1}}
# b = o_m - k + 1

for u_mk in u_list:

    m = u_mk.machine
    k = u_mk.k

    m_op_list = OM[M.index(m)]

    q_max = math.ceil(math.log2(k + 1))
    M_u = 2**q_max
        
    b = len(m_op_list) - k + 1
    vars = []
    a = []

    for o in m_op_list:
        vars.append(bz(L(o)))
        a.append(1)
        
    vars.append(u_mk)
    a.append(M_u)

    for q in range(q_max):
        vars.append(slack("6", u_mk, q))
        a.append(-2**q)

    Q_C6.extend(linear_constraint(x, vars, a, b))

P_C6 = 2000
# print(Q_C6)
# update_Q(Q, Q_C6, P_C6)
# print_Q_nonZero(Q,x)
      



#### 4.6.1) X columns with only one "1" or none
  $$ \sum_{jps} { X^m_{jps,k} } = u_{mk} $$
    for all m and k


In [671]:
Q_C6_1 = []

# ( sum_jps {X^m_(jps,k)} - u_mk )^2

for u_mk in u_list:

    m = u_mk.machine
    k = u_mk.k

    m_op_list = OM[M.index(m)]
        
    vars = []
    a = []

    for o in m_op_list:
        vars.append(X(o,k))
        a.append(1)
        
    vars.append(u_mk)
    a.append(-1)


    Q_C6_1.extend(linear_constraint(x, vars, a))

P_C6_1 = 2000
print(Q_C6_1)
# update_Q(Q, Q_C6_1, P_C6_1)
# print_Q_nonZero(Q,x)

[[82, 82, 1], [82, 83, 1], [82, 84, 1], [82, 85, 1], [82, 86, 1], [82, 112, -1], [83, 83, 1], [83, 84, 1], [83, 85, 1], [83, 86, 1], [83, 112, -1], [84, 84, 1], [84, 85, 1], [84, 86, 1], [84, 112, -1], [85, 85, 1], [85, 86, 1], [85, 112, -1], [86, 86, 1], [86, 112, -1], [112, 112, 1], [87, 87, 1], [87, 88, 1], [87, 89, 1], [87, 90, 1], [87, 91, 1], [87, 113, -1], [88, 88, 1], [88, 89, 1], [88, 90, 1], [88, 91, 1], [88, 113, -1], [89, 89, 1], [89, 90, 1], [89, 91, 1], [89, 113, -1], [90, 90, 1], [90, 91, 1], [90, 113, -1], [91, 91, 1], [91, 113, -1], [113, 113, 1], [92, 92, 1], [92, 93, 1], [92, 94, 1], [92, 95, 1], [92, 96, 1], [92, 114, -1], [93, 93, 1], [93, 94, 1], [93, 95, 1], [93, 96, 1], [93, 114, -1], [94, 94, 1], [94, 95, 1], [94, 96, 1], [94, 114, -1], [95, 95, 1], [95, 96, 1], [95, 114, -1], [96, 96, 1], [96, 114, -1], [114, 114, 1], [97, 97, 1], [97, 98, 1], [97, 99, 1], [97, 100, 1], [97, 101, 1], [97, 115, -1], [98, 98, 1], [98, 99, 1], [98, 100, 1], [98, 101, 1], [98, 115


<h3>4.7) Precedence restriction</h3>

4.7.1) By precedence
  S_jp'm's' >= S_jpms + L_jpms    //for all precedences pms -> p'm's'

  S_jp'm's' = S_jpms + L_jpms + slack(4qb or less)



4.7.1.1) Final dummy operation
  E_j >= S_j(p_j)ms + L_j(p_j)ms

  E_j = S_j(p_j)ms + L_j(p_j)ms + slack(4qb or less)



4.7.2) Alternatively (+M)
  S_jp'm's' >= S_jpms + L_jpms - M * bz_jpms   //for all precedences pms -> p'm's'
  S_jp'm's' = S_jpms + L_jpms - M * bz_jpms + slack
  -> slack_max = max{ S' - S + M } = 15 + M ~ 2*M (5qb)

  S' >= S - M
  M >= max{S-S'} = 15 -> M = 15 o 16



4.7.2.1) Final dummy operation
  E_j >= S_j(p_j)ms + L_j(p_j)ms - M * bz_j(p_j)ms    for all m, s
  E_j = S_j(p_j)ms + L_j(p_j)ms - M * bz_j(p_j)ms + slack
  -> slack_max = max{ E - S + M } = 15 + M ~ 2*M (5qb)

  M >= S - E iff M >= max{S-E} = 15 -> M = 15 or 16
  
  (*) si para un E_j hay solo una restriccion (una unica tarea final), la desigualdad pasa a ser igualdad y se fija la slack como 0
            (((( fix SLACK = 0 ))))



#### 4.7.3) A possible reinforcing constraint

$$ \sum_{k} {X^{m}_{(jp's'),k} \times k} + M \times bz_{jp'ms'} \geq \sum_{k} {X^m_{(jps)k}\times k} - M\times bz_{jpms} $$



<h3>4.8) Non-superposition of operations:</h3>
for each machine m, k = 2,...o_m

    sum_jps{
      S_jpms * X^m_(jps,k)
    }

    \>= sum_jps {
      ( S_jpms + Ljpms ) * X^m_(jps,k-1)
    }

    \+ sum_(jps, j'p's') {
      X^m_(jps,k-1) * TT^m_p,p' * X^m_(j'p's',k)
    }
    
    \- M * (1 - u_mk)




4.8.1) Avoiding the quadratic terms
  sum_jps{
    y^m_(jps,k)
  }
\>= sum_jps {
    y^m_(jps,k-1) + Ljpms * z^m_(jps,k-1)
  }
\+ sum_(jps, j'p's') {
    TT^m_p,p' w^m_(jps,j'p's',k-1)
  }
\- M * (1 - u_mk)




<h3>4.9) Quadratic constraint auxiliary variables</h3>



4.9.1) y^m_ikq = X^m_(i,k) * {S_i}_q
  P * ( 3*y^m_ikq + X^m_(i,k)*{S_i}_q - 2*{S_i}_q*y^m_ikq - 2*X^m_(i,k)*y^m_ikq )



4.9.2) z^m_ikq = X^m_(i,k) * {L_i}_q
  P * ( 3*z^m_ikq + X^m_(i,k)*{L_i}_q - 2*{L_i}_q*z^m_ikq - 2*X^m_(i,k)*z^m_ikq )



4.9.3) w^m_ijk = X^m_(i,k) * X^m_(i',k+1)
  P * ( 3*w^m_ijk + X^m_(i,k)*X^m_(j,k+1) - 2*X^m_(j,k+1)*w^m_ijk -2*X^m_(i,k)*w^m_ijk )


In [672]:
# sampleset = sendToDwave(Q, 10)
# print(sampleset)
# samples_dict = count_elements(sampleset, len(Q))
# make_histogram(samples_dict)

